# Pillar 3 · Agent Deployment and Monitoring

This notebook orchestrates deployment readiness reviews and sets up continuous monitoring for Azure AI agents.

## Learning Objectives

- Standardize deployment checklists for new or updated agents
- Configure telemetry pipelines for real-time monitoring
- Define rollback and incident response procedures

## Deployment Playbook

1. **Pre-Deployment Validation**: Re-run functional tests outlined in `01_agent/01_foundry_agent.ipynb`, capturing artifacts per agent release.
2. **Observability Hooks**: Extend the run management utilities in `agent_utils.AgentManager.run_agent` to emit metrics to Azure Monitor.
3. **Safety Gates**: Require dual-approval for production pushes, logging decisions via `AgentDB.update_agent` with status transitions.
4. **Incident Workflow**: Document escalation paths and rollback steps referencing Azure DevOps release pipelines.

## Monitoring Checklist

- Latency, accuracy, and safety KPIs defined
- Alert thresholds and notification channels configured
- Automated rollback tested and documented

## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

# Replace with the directory you want to add
new_path_entry = "/opt/homebrew/bin"
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

In [ ]:
# Ensure the agent manager is available for deployment checks
import os
import sys
import time
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_utils import AgentManager

# Load environment variables
load_dotenv("../.env")

print("✅ Imports successful")

endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
if not endpoint:
    raise ValueError("Set AZURE_AI_PROJECT_ENDPOINT in .env.local before running this notebook.")

project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

manager = AgentManager(project_client)
print("Agent manager ready.")

In [ ]:
# Run a deployment readiness sweep and capture latency metrics
agent_id = "asst_CncymMdTqov5hRCQQJrvLwhX"  # Replace with actual agent ID

if not agent_id:
    raise ValueError(
        "Set TARGET_AGENT_ID in your environment to the agent under review.")

test_prompts = [
    "Summarize the latest release notes for our marketing team.",
    "Give troubleshooting steps for a customer facing login issues.",
    "Escalate a high-severity incident to the on-call engineer.",
    "Provide a compliance-safe response to a sensitive HR inquiry."
]

thread = manager.create_thread({"purpose": "release-validation"})
run_metrics = []

for prompt in test_prompts:
    manager.add_message(thread.id, prompt, role="user")
    started = time.time()
    response = manager.run_agent_simple(
        thread_id=thread.id,
        agent_id=agent_id,
        user_message=prompt,
        verbose=False,
        temperature=0.3
    )
    latency = time.time() - started
    run_metrics.append({
        "prompt": prompt,
        "latency_seconds": latency,
        "response": response
    })
    print(f"{prompt[:40]}... -> {latency:.2f}s")

In [ ]:
# Persist release validation results to Cosmos DB metadata
from datetime import datetime
from agent_db import AgentDB

db = AgentDB() 
release_evidence = {
    "runId": f"release-{datetime.utcnow().isoformat()}Z",
    "metrics": run_metrics
}

db.update_agent(
    azure_agent_id=agent_id,
    lastReleaseValidation=release_evidence
)
print("Stored release validation evidence in Cosmos DB.")

## Next Notebook

Move to `05_auditability_traceability.ipynb` to reinforce audit-ready practices.